In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO

url = 'https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/credential_access/host/empire_mimikatz_backupkeys_dcerpc_smb_lsarpc.zip'
zipFileRequest = requests.get(url)
zipFile = ZipFile(BytesIO(zipFileRequest.content))
datasetJSONPath = zipFile.extract(zipFile.namelist()[0])

In [2]:
import pandas as pd
from pandas.io import json

df = json.read_json(path_or_buf=datasetJSONPath, lines=True)

C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
(
df[['@timestamp','Hostname','ObjectServer','ObjectType','ObjectName']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4662)
    & (df['AccessMask'] == '0x2')
    & (df['Message'].str.lower().str.contains('.*bckupkey.*', regex=True))
]
.head()
)

,@timestamp,Hostname,ObjectServer,ObjectType,ObjectName
5279,2020-10-22T18:33:36.923Z,MORDORDC.theshire.local,LSA,SecretObject,Policy\Secrets\G$BCKUPKEY_PREFERRED
5283,2020-10-22T18:33:36.923Z,MORDORDC.theshire.local,LSA,SecretObject,Policy\Secrets\G$BCKUPKEY_a0feda20-878f-4e68-b...
5287,2020-10-22T18:33:36.923Z,MORDORDC.theshire.local,LSA,SecretObject,Policy\Secrets\G$BCKUPKEY_P
5291,2020-10-22T18:33:36.923Z,MORDORDC.theshire.local,LSA,SecretObject,Policy\Secrets\G$BCKUPKEY_0343c16b-26f1-4e2c-8...


In [6]:
backupKeyDf = (
# (
df[['@timestamp','Hostname','ObjectName','IpAddress','SubjectLogonId']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4662)
    & (df['AccessMask'] == '0x2')
    & (df['ObjectName'].str.lower().str.contains('.*bckupkey.*', regex=True))
]
.head()
)

networkLogonDf = (
# (
df[['@timestamp','Hostname','TargetUserName','TargetLogonId','IpAddress']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4624)
    & (df['LogonType'] == 3)
    & (~df['SubjectUserName'].str.endswith('.*$', na=False))
]
.head()
)

(
pd.merge(backupKeyDf, networkLogonDf,
    left_on = ['SubjectLogonId','Hostname'], right_on = ['TargetLogonId','Hostname'], how = 'inner')
)

,@timestamp_x,Hostname,ObjectName,IpAddress_x,SubjectLogonId,@timestamp_y,TargetUserName,TargetLogonId,IpAddress_y
0,2020-10-22T18:33:36.923Z,MORDORDC.theshire.local,Policy\Secrets\G$BCKUPKEY_PREFERRED,NaN,0x2b7e3c0,2020-10-22T18:33:36.922Z,pgustavo,0x2b7e3c0,172.18.39.5
1,2020-10-22T18:33:36.923Z,MORDORDC.theshire.local,Policy\Secrets\G$BCKUPKEY_a0feda20-878f-4e68-b...,NaN,0x2b7e3c0,2020-10-22T18:33:36.922Z,pgustavo,0x2b7e3c0,172.18.39.5
2,2020-10-22T18:33:36.923Z,MORDORDC.theshire.local,Policy\Secrets\G$BCKUPKEY_P,NaN,0x2b7e3c0,2020-10-22T18:33:36.922Z,pgustavo,0x2b7e3c0,172.18.39.5
3,2020-10-22T18:33:36.923Z,MORDORDC.theshire.local,Policy\Secrets\G$BCKUPKEY_0343c16b-26f1-4e2c-8...,NaN,0x2b7e3c0,2020-10-22T18:33:36.922Z,pgustavo,0x2b7e3c0,172.18.39.5


In [6]:
(
df[['@timestamp','Hostname','SubjectUserName','ShareName','RelativeTargetName']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 5145)
    & (df['RelativeTargetName'].str.lower() == 'protected_storage')
    & (df['ShareName'].str.contains('.*IPC.*', regex=True))
]
)

,@timestamp,Hostname,SubjectUserName,ShareName,RelativeTargetName


In [7]:
(
df[['@timestamp','Hostname','SubjectUserName']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4692)
]
)

,@timestamp,Hostname,SubjectUserName
